In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.table import Table

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.dpi'] = 150


In [ ]:
data_dir = Path('/Users/marchuertascompany/Documents/data/COSMOS-Web/catalogs')
catalog_path = data_dir / 'COSMOSWeb_mastercatalog_v1.fits'

if not catalog_path.exists():
    raise FileNotFoundError(f"Catalog not found at {catalog_path}")

catalog_path


In [ ]:
photom_cols = ['id', 'warn_flag', 'mag_model_f444w', 'flag_star_hsc']
lephare_cols = ['type', 'zfinal', 'mass_med', 'ssfr_med', 'mabs_nuv', 'mabs_r', 'mabs_j']

cat_photom = Table.read(catalog_path, hdu=1)[photom_cols]
lephare_raw = Table.read(catalog_path, hdu=2)
cat_lephare = lephare_raw[lephare_cols]

catalog = cat_lephare.copy()
for col in photom_cols:
    catalog[col] = cat_photom[col]

catalog


In [ ]:
# Load binary disturbed/not-disturbed probabilities
morph_pred_path = Path('/Users/marchuertascompany/Documents/data/COSMOS-Web/zoobot/ilbert/ilbert_visual_zoobot_morphology.fits')
binary_cols = ['binary_NOT_DISTURBED', 'binary_DISTURBED']

if not morph_pred_path.exists():
    raise FileNotFoundError(f"Morphology catalog not found at {morph_pred_path}")

binary_table = Table.read(morph_pred_path)
binary_df = binary_table.to_pandas()


def normalize_ids(values):
    series = pd.Series(values).astype(str).str.strip()
    series = series.str.replace(r'\.0$', '', regex=True)
    return series


binary_df['id'] = normalize_ids(binary_df['id'])
catalog_ids = normalize_ids(catalog['id'])

binary_matrix = (
    binary_df.set_index('id')
    .reindex(catalog_ids)[binary_cols]
    .to_numpy()
)

for idx, col in enumerate(binary_cols):
    values = np.asarray(binary_matrix[:, idx], dtype=float)
    catalog[col] = values

binary_available = np.isfinite(binary_matrix).any(axis=1)
catalog['has_any_binary_morphology'] = binary_available

valid_count = int(binary_available.sum())
print(f"Binary morphology available for {valid_count:,} sources ({valid_count/len(catalog):.1%}).")


In [ ]:
# Compute rest-frame color combinations
mabs_nuv = np.ma.filled(np.asarray(catalog['mabs_nuv'], dtype=float), np.nan)
mabs_r = np.ma.filled(np.asarray(catalog['mabs_r'], dtype=float), np.nan)
mabs_j = np.ma.filled(np.asarray(catalog['mabs_j'], dtype=float), np.nan)
mass_log = np.ma.filled(np.asarray(catalog['mass_med'], dtype=float), np.nan)
z = np.asarray(catalog['zfinal'], dtype=float)

nuv_minus_r = mabs_nuv - mabs_r
r_minus_j = mabs_r - mabs_j

catalog['nuv_minus_r'] = nuv_minus_r
catalog['r_minus_j'] = r_minus_j

# Clean-galaxy mask from the tutorial
clean_mask = (
    (np.asarray(catalog['type']) == 0) &
    (np.asarray(catalog['warn_flag']) == 0) &
    (np.abs(np.asarray(catalog['mag_model_f444w'])) < 30) &
    (np.asarray(catalog['flag_star_hsc']) == 0)
)

# Require finite colors and masses
finite_mask = (
    np.isfinite(nuv_minus_r) &
    np.isfinite(r_minus_j) &
    np.isfinite(mass_log)
)

clean_mask &= finite_mask

# Low-mass and quiescent requirements
low_mass_mask = mass_log < 10.0
quiescent_mask = (nuv_minus_r > 3.1) & (nuv_minus_r > 3.0 * r_minus_j + 1.0)

final_mask = clean_mask & low_mass_mask & quiescent_mask

quiescent_sample = catalog[final_mask]
print(f"Total catalog sources        : {len(catalog):,}")
print(f"Clean galaxy sample          : {clean_mask.sum():,}")
print(f"Low-mass quiescent selection : {len(quiescent_sample):,}")


In [ ]:
# Binary morphology classification
binary_cols = ['binary_NOT_DISTURBED', 'binary_DISTURBED']
binary_labels = np.array(['not-disturbed', 'disturbed'])

binary_scores = np.column_stack([
    np.asarray(catalog[col], dtype=float) for col in binary_cols
])

binary_scores_for_sort = np.nan_to_num(binary_scores, nan=-np.inf)
binary_order = np.argsort(binary_scores_for_sort, axis=1)
binary_main_idx = binary_order[:, -1]

binary_counts = np.isfinite(binary_scores).sum(axis=1)
binary_main_class = np.full(len(catalog), 'unclassified', dtype='<U16')
binary_main_score = np.full(len(catalog), np.nan)
binary_mask = binary_counts > 0
binary_main_class[binary_mask] = binary_labels[binary_main_idx[binary_mask]]
binary_main_score[binary_mask] = binary_scores[np.arange(len(catalog)), binary_main_idx][binary_mask]

catalog['binary_main_class'] = binary_main_class
catalog['binary_main_score'] = binary_main_score
catalog['binary_unique_class'] = binary_main_class

print('Binary morphology counts (all sources with classifications):')
for label in binary_labels:
    count = np.count_nonzero(binary_main_class == label)
    if count > 0:
        print(f"  {label:12s}: {count:,}")

print('Binary morphology counts within the selected sample:')
selected_binary = final_mask & catalog['has_any_binary_morphology']
unique_labels, unique_counts = np.unique(catalog['binary_unique_class'][selected_binary], return_counts=True)
for label, count in zip(unique_labels, unique_counts):
    print(f"  {label:12s}: {count:,}")


In [ ]:
# UVJ diagram colored by binary morphology class
z_bins = np.arange(0.0, 5.6, 0.5)
z_labels = [f"{z_bins[i]:.1f} ≤ z < {z_bins[i+1]:.1f}" for i in range(len(z_bins) - 1)]

binary_palette = {
    'not-disturbed': '#4C72B0',
    'disturbed': '#C44E52'
}

binary_quiescent_mask = final_mask & catalog['has_any_binary_morphology']

fig, axes = plt.subplots(3, 4, figsize=(14, 8), sharex=True, sharey=True)
axes = axes.ravel()

for idx, (z_min, z_max) in enumerate(zip(z_bins[:-1], z_bins[1:])):
    ax = axes[idx]
    bin_mask = clean_mask & (catalog['zfinal'] >= z_min) & (catalog['zfinal'] < z_max)
    quiescent_bin_mask = binary_quiescent_mask & (catalog['zfinal'] >= z_min) & (catalog['zfinal'] < z_max)

    if bin_mask.sum() == 0:
        ax.text(0.5, 0.5, 'No data', ha='center', va='center')
        ax.set_title(z_labels[idx])
        continue

    ax.hexbin(
        r_minus_j[bin_mask],
        nuv_minus_r[bin_mask],
        gridsize=80,
        extent=(-1, 3, -1, 8),
        cmap='Greys',
        mincnt=1,
        linewidths=0
    )

    classes = catalog['binary_unique_class'][quiescent_bin_mask]
    colors = [binary_palette.get(cls, '#333333') for cls in classes]

    ax.scatter(
        r_minus_j[quiescent_bin_mask],
        nuv_minus_r[quiescent_bin_mask],
        s=12,
        c=colors,
        linewidths=0.25,
        edgecolor='white',
        alpha=0.9
    )

    ax.plot([-1, 0.7], [3.1, 3.1], color='navy', linestyle='--', linewidth=1.0)
    rj_line = np.linspace(0.7, 3.0, 100)
    ax.plot(rj_line, 3.0 * rj_line + 1.0, color='navy', linestyle='--', linewidth=1.0)

    ax.set_title(z_labels[idx])
    ax.set_xlim(-1, 3)
    ax.set_ylim(-1, 8)

for ax in axes[8:]:
    ax.set_xlabel(r'$r - J$')
for ax in axes[::4]:
    ax.set_ylabel(r'$NUV - r$')

handles = []
for label in sorted(set(catalog['binary_unique_class'][binary_quiescent_mask])):
    color = binary_palette.get(label, '#333333')
    handles.append(plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=8, label=label))

fig.legend(handles=handles, loc='upper center', ncol=2, frameon=False, bbox_to_anchor=(0.5, 1.02))
fig.suptitle('Binary morphology class of quiescent galaxies', y=0.98)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:
# UVJ diagram with binary classes (aggregated panel)
fig, axes = plt.subplots(3, 4, figsize=(14, 8), sharex=True, sharey=True)
axes = axes.ravel()

for idx, (z_min, z_max) in enumerate(zip(z_bins[:-1], z_bins[1:])):
    ax = axes[idx]
    bin_mask = clean_mask & (catalog['zfinal'] >= z_min) & (catalog['zfinal'] < z_max)
    quiescent_bin_mask = binary_quiescent_mask & (catalog['zfinal'] >= z_min) & (catalog['zfinal'] < z_max)

    if bin_mask.sum() == 0:
        ax.text(0.5, 0.5, 'No data', ha='center', va='center')
        ax.set_title(z_labels[idx])
        continue

    ax.hexbin(
        r_minus_j[bin_mask],
        nuv_minus_r[bin_mask],
        gridsize=80,
        extent=(-1, 3, -1, 8),
        cmap='Greys',
        mincnt=1,
        linewidths=0
    )

    classes = catalog['binary_unique_class'][quiescent_bin_mask]
    colors = [binary_palette.get(cls, '#333333') for cls in classes]

    ax.scatter(
        r_minus_j[quiescent_bin_mask],
        nuv_minus_r[quiescent_bin_mask],
        s=2,
        c=colors,
        linewidths=0.25,
        edgecolor='white',
        alpha=0.8
    )

    ax.plot([-1, 0.7], [3.1, 3.1], color='navy', linestyle='--', linewidth=1.0)
    rj_line = np.linspace(0.7, 3.0, 100)
    ax.plot(rj_line, 3.0 * rj_line + 1.0, color='navy', linestyle='--', linewidth=1.0)

    ax.set_title(z_labels[idx])
    ax.set_xlim(-1, 3)
    ax.set_ylim(-1, 8)

for ax in axes[8:]:
    ax.set_xlabel(r'$r - J$',fontsize=20)
for ax in axes[::4]:
    ax.set_ylabel(r'$NUV - r$',fontsize=20)

handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=binary_palette.get(cls, '#333333'),
                      markersize=8, label=cls) for cls in sorted(set(catalog['binary_unique_class'][binary_quiescent_mask]))]

fig.legend(handles=handles, loc='upper center', ncol=2, frameon=False, bbox_to_anchor=(0.5, 1.02),fontsize=20)
fig.suptitle('Binary morphology quiescent galaxies ($\log M_*/M_\odot<10$)', y=0.94,fontsize=20)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig('plots/lowmass_Q_binary_NUV_J.png')
plt.show()


In [ ]:
# Binary morphology fractions vs density
binary_density_mask = final_mask & catalog['has_any_binary_morphology'] & np.isfinite(catalog['overdensity'])

if binary_density_mask.sum() == 0:
    print('No density information available for the selected sample.')
else:
    redshift_bins_density = [
        (0.0, 1.0, '0 ≤ z < 1'),
        (1.0, 3.0, '1 ≤ z < 3'),
        (3.0, 5.5, '3 ≤ z < 5.5')
    ]
    num_quantiles = 4
    quantile_labels = [f'Q{i+1}' for i in range(num_quantiles)]
    overall_classes = ['not-disturbed', 'disturbed']

    fig, axes = plt.subplots(1, len(redshift_bins_density), figsize=(16, 4.5), sharey=True)
    if len(redshift_bins_density) == 1:
        axes = [axes]

    positions = np.arange(num_quantiles)

    for ax, (z_min, z_max, title) in zip(axes, redshift_bins_density):
        bin_mask = binary_density_mask & (catalog['zfinal'] >= z_min) & (catalog['zfinal'] < z_max)
        dens_values = np.asarray(catalog['overdensity'][bin_mask], dtype=float)
        morph_classes = np.asarray(catalog['binary_unique_class'][bin_mask])

        valid = np.isfinite(dens_values)
        dens_values = dens_values[valid]
        morph_classes = morph_classes[valid]

        if dens_values.size == 0:
            ax.text(0.5, 0.5, 'No data', ha='center', va='center')
            ax.set_title(title)
            ax.set_xticks(positions)
            ax.set_xticklabels(quantile_labels)
            ax.set_ylim(0, 1)
            continue

        n = dens_values.size
        ranks = np.argsort(np.argsort(dens_values))
        quantile_indices = np.floor(ranks / n * num_quantiles).astype(int)
        quantile_indices[quantile_indices == num_quantiles] = num_quantiles - 1

        totals = np.array([np.count_nonzero(quantile_indices == q) for q in range(num_quantiles)], dtype=float)
        bottom = np.zeros(num_quantiles)

        for cls in overall_classes:
            counts = np.array([
                np.count_nonzero((quantile_indices == q) & (morph_classes == cls))
                for q in range(num_quantiles)
            ], dtype=float)
            fractions = np.zeros(num_quantiles, dtype=float)
            valid_totals = totals > 0
            fractions[valid_totals] = counts[valid_totals] / totals[valid_totals]
            ax.bar(
                positions,
                fractions,
                bottom=bottom,
                width=0.65,
                color=binary_palette.get(cls, '#333333'),
                label=cls
            )
            bottom += fractions

        ax.set_ylim(0, 1)
        ax.set_xticks(positions)
        ax.set_xticklabels(quantile_labels)
        ax.set_xlabel('Density quantile')
        ax.set_title(title)
        ax.grid(axis='y', linestyle=':', alpha=0.4)
        sample_size = int(totals.sum())
        ax.text(0.5, -0.12, f'N={sample_size}', transform=ax.transAxes, ha='center', va='top')

    axes[0].set_ylabel('Morphology fraction')
    handles = []
    for cls in overall_classes:
        color = binary_palette.get(cls, '#333333')
        handles.append(plt.Line2D([0], [0], marker='s', color=color, linestyle='', markersize=8, label=cls))
    axes[-1].legend(handles=handles, loc='upper left', bbox_to_anchor=(1.02, 1), frameon=False)
    plt.tight_layout(rect=[0, 0, 0.88, 1])
    plt.show()


In [ ]:
# SFH by binary morphology classes
binary_density_mask = final_mask & catalog['has_any_binary_morphology'] & np.isfinite(catalog['overdensity'])

if binary_density_mask.sum() == 0:
    print('No density information available for the selected sample.')
else:
    redshift_bins_density = [
        (0.0, 1.0, '0 ≤ z < 1'),
        (1.0, 3.0, '1 ≤ z < 3'),
        (3.0, 5.5, '3 ≤ z < 5.5')
    ]
    num_quantiles = 4
    density_quantiles = [('Lowest (Q1)', 0), ('Highest (Q4)', num_quantiles - 1)]

    aggregated_labels = np.array(catalog['binary_unique_class'])
    aggregated_classes = ['not-disturbed', 'disturbed']

    time_array = np.column_stack([catalog[col] for col in sfh_time_cols]) / 1e3  # convert to Gyr
    sfr_fraction_array = np.column_stack([catalog[col] for col in sfh_sfr_cols])
    sfh_integrated = np.asarray(catalog['sfh_integrated'], dtype=float)
    sfr_array = sfr_fraction_array * sfh_integrated[:, None]

    valid_time = time_array[np.isfinite(time_array)]
    valid_sfr = sfr_array[np.isfinite(sfr_array)]
    if valid_time.size == 0 or valid_sfr.size == 0:
        print('SFH information missing for all selected sources.')
    else:
        time_min = max(valid_time.min(), 1e-3)
        time_max = valid_time.max()

        fig, axes = plt.subplots(len(redshift_bins_density), len(density_quantiles), figsize=(12, 9), sharex=True, sharey=True)
        axes = np.atleast_2d(axes)

        for i, (z_min, z_max, title) in enumerate(redshift_bins_density):
            bin_mask = binary_density_mask & (catalog['zfinal'] >= z_min) & (catalog['zfinal'] < z_max)
            if not bin_mask.any():
                for ax in axes[i]:
                    ax.text(0.5, 0.5, 'No data', ha='center', va='center')
                    ax.set_title(f'{title}(no sample)')
                continue

            bin_indices = np.where(bin_mask)[0]
            dens_values = np.asarray(catalog['overdensity'][bin_mask], dtype=float)
            valid = np.isfinite(dens_values)
            bin_indices = bin_indices[valid]
            dens_values = dens_values[valid]

            if dens_values.size == 0:
                for ax in axes[i]:
                    ax.text(0.5, 0.5, 'No density', ha='center', va='center')
                    ax.set_title(f'{title}(no density)')
                continue

            ranks = np.argsort(np.argsort(dens_values))
            quantile_indices = np.floor(ranks / dens_values.size * num_quantiles).astype(int)
            quantile_indices[quantile_indices == num_quantiles] = num_quantiles - 1

            for j, (label, q_idx) in enumerate(density_quantiles):
                ax = axes[i, j]
                selected_indices = bin_indices[quantile_indices == q_idx]
                if selected_indices.size == 0:
                    ax.text(0.5, 0.5, 'No data', ha='center', va='center')
                    ax.set_title(f'{title}{label}')
                    continue

                lines_drawn = 0
                for cls in aggregated_classes:
                    class_mask = (aggregated_labels == cls)
                    group_mask = class_mask.copy()
                    group_mask[:] = False
                    group_mask[selected_indices] = True
                    group_mask &= class_mask

                    sfh_subset = sfr_array[group_mask]
                    time_subset = time_array[group_mask]

                    if sfh_subset.size == 0:
                        continue

                    finite_rows = (
                        np.isfinite(sfh_subset).all(axis=1) &
                        np.isfinite(time_subset).all(axis=1)
                    )
                    if finite_rows.sum() == 0:
                        continue

                    mean_time = np.nanmean(time_subset[finite_rows], axis=0)
                    mean_sfr = np.nanmean(sfh_subset[finite_rows], axis=0)

                    ax.step(mean_time, mean_sfr, where='mid', color=binary_palette.get(cls, '#333333'), label=cls)
                    lines_drawn += 1

                if lines_drawn == 0:
                    ax.text(0.5, 0.5, 'No SFH data', ha='center', va='center')

                ax.set_title(f'{title} {label}')
                ax.set_xscale('log')
                ax.set_yscale('log')
                ax.set_xlim(time_min, time_max)

        for ax in axes[:, 0]:
            ax.set_ylabel(r'$\mathrm{SFR}$ [M$_{\odot}$ yr$^{-1}$]',fontsize=20)
        for ax in axes[-1, :]:
            ax.set_xlabel('Lookback time [Gyr]',fontsize=20)

        handles = [plt.Line2D([0], [0], color=binary_palette.get(cls, '#333333'), label=cls) for cls in aggregated_classes]
        axes[0, -1].legend(handles=handles, loc='upper left', bbox_to_anchor=(1.02, 1), frameon=False,fontsize=20)

        plt.tight_layout(rect=[0, 0, 0.88, 1])
        plt.savefig('plots/SFH_CIGALE_binary_lowmassQ.png')
        plt.show()


In [ ]:
# Binary morphology fractions by redshift bin (no density split)
z_bins = np.arange(0.0, 5.6, 0.5)
z_labels = [f"{z_bins[i]:.1f} ≤ z < {z_bins[i+1]:.1f}" for i in range(len(z_bins) - 1)]

binary_palette = {
    'not-disturbed': '#4C72B0',
    'disturbed': '#C44E52'
}

binary_quiescent_mask = final_mask & catalog['has_any_binary_morphology']

positions = np.arange(len(z_labels))
bottom = np.zeros(len(z_labels))

fig, ax = plt.subplots(1, 1, figsize=(14, 4.5))

for cls in ['not-disturbed', 'disturbed']:
    counts = []
    for z_min, z_max in zip(z_bins[:-1], z_bins[1:]):
        bin_mask = binary_quiescent_mask & (catalog['zfinal'] >= z_min) & (catalog['zfinal'] < z_max)
        total = np.count_nonzero(bin_mask)
        if total == 0:
            counts.append(0.0)
        else:
            cls_count = np.count_nonzero(catalog['binary_unique_class'][bin_mask] == cls)
            counts.append(cls_count / total)
    fractions = np.array(counts)
    ax.bar(
        positions,
        fractions,
        bottom=bottom,
        width=0.7,
        color=binary_palette.get(cls, '#333333'),
        label=cls
    )
    bottom += fractions

ax.set_xticks(positions)
ax.set_xticklabels(z_labels, rotation=45, ha='right')
ax.set_ylim(0, 1)
ax.set_ylabel('Morphology fraction')
ax.set_xlabel('Redshift bin')
ax.grid(axis='y', linestyle=':', alpha=0.4)

handles = [
    plt.Line2D([0], [0], marker='s', color=binary_palette.get(cls, '#333333'),
               linestyle='', markersize=8, label=cls)
    for cls in ['not-disturbed', 'disturbed']
]
ax.legend(handles=handles, loc='upper left', bbox_to_anchor=(1.02, 1), frameon=False)

plt.tight_layout(rect=[0, 0, 0.88, 1])
plt.show()
